# Podział dokumentów na fragmenty

* W wektorowej bazie danych będę przechowywać wektory zanurzeń dla fragmentów dokumentów.
* Model zanurzeń ma ograniczenie co do liczby tokenów (np. 256 w all-MiniLM-L6-v2)
* Razem z framgentem zapisywać będę metadane (z którego pliku, jaki kurs, jaki tytuł, jaki fragment dokumentu, numer fragmentu w dokumencie)
* W moim przypadku dokumenty to pliki markdown
* Mogę zastosować podział w oparciu o konstrukcje składniowe markdowna
  * najpierw nagłówki
  * potem akapity
  * potem linie
  * jeśli fragmenty dalej będą za długie to dalej podział rekursywny
* LangChain implementuje narzędzia do podziału

https://medium.com/@jesvinkjustin/from-zero-to-rag-the-art-of-document-chunking-and-embedding-for-rag-d9764695cc46

In [16]:
from langchain_text_splitters import MarkdownHeaderTextSplitter, RecursiveCharacterTextSplitter

In [17]:
with open("../data/raw/notes/inzynieria-uczenia-maszynowego/02-zadania-modelowania.md") as f:
    document = f.read()

document

'# Definiowanie zadań modelowania\n\n## Kluczowe elementy\n* Jakie jest zadanie biznesowe / domenowe\n\t* czy je dostatecznie dobrze rozumiemy\n\t* jakie są ograniczenia (czasowe, zasobów, formalne, ...)\n* Jakie są zadania modelowania\n\t* czy są odpowiednie dane do budowy modeli\n\t* czy rozumiemy proces, który je wygenerował\n* Jakie są kryteria sukcesu\n\t* na poziomie zadań modelowania (analityczne)\n\t* biznesowe\n\n## Formułowanie zadania biznesowego\n* Potrzebny jest ekspert domenowy\n\t* np. product owner w metodologiach zwinnych\n\t* dobrze rozumie zagadnienie, którym mamy się zająć i wynikające z niego potrzeby\n\t* zna specyfikę pracy nad modelowaniem danych\n\t* jest osobą decyzyjną w zakresie projektu - wspólne ustalenia są wiążące\n* Analizujemy kontekst w jakim występuje potrzeba biznesowa\n\t* jaka jest obecna sytuacja\n\t* co ma zostać\xa0wprowadzone / zmienione\n\t* jakie właściwości / cechy powinno mieć docelowe rozwiązanie\n\t* jakie są założenia, oczekiwania, ogra

Maksymalna długość fragmentu - przyjmuję 250 znaków
(trochę mniej niż max tokenów w modelu, model daje 1 token na 1 znak w jezyku polskim)

In [18]:
MAX_CHUNK_LENGTH = 250

In [19]:
headers_to_split = [
    ("#", "H1"),
    ("##", "H2"),
    ("###", "H3"),
    ("####", "H4"),
]
header_splitter = MarkdownHeaderTextSplitter(headers_to_split)

In [20]:
markdown_splits = header_splitter.split_text(document)
len(markdown_splits)

21

In [21]:
markdown_splits[0]

Document(metadata={'H1': 'Definiowanie zadań modelowania', 'H2': 'Kluczowe elementy'}, page_content='* Jakie jest zadanie biznesowe / domenowe\n* czy je dostatecznie dobrze rozumiemy\n* jakie są ograniczenia (czasowe, zasobów, formalne, ...)\n* Jakie są zadania modelowania\n* czy są odpowiednie dane do budowy modeli\n* czy rozumiemy proces, który je wygenerował\n* Jakie są kryteria sukcesu\n* na poziomie zadań modelowania (analityczne)\n* biznesowe')

In [22]:
markdown_splits[1]

Document(metadata={'H1': 'Definiowanie zadań modelowania', 'H2': 'Formułowanie zadania biznesowego'}, page_content='* Potrzebny jest ekspert domenowy\n* np. product owner w metodologiach zwinnych\n* dobrze rozumie zagadnienie, którym mamy się zająć i wynikające z niego potrzeby\n* zna specyfikę pracy nad modelowaniem danych\n* jest osobą decyzyjną w zakresie projektu - wspólne ustalenia są wiążące\n* Analizujemy kontekst w jakim występuje potrzeba biznesowa\n* jaka jest obecna sytuacja\n* co ma zostaćwprowadzone / zmienione\n* jakie właściwości / cechy powinno mieć docelowe rozwiązanie\n* jakie są założenia, oczekiwania, ograniczenia, zasoby')

In [23]:
splt_lengths = [len(fragment.page_content) for fragment in markdown_splits]
print(max(splt_lengths))
print(min(splt_lengths))
print(sum(splt_lengths) / len(splt_lengths))

807
56
337.5238095238095


Otrzymane podziały mogą być za długie

In [26]:
md_splitter = MarkdownHeaderTextSplitter(headers_to_split, strip_headers=False)
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", " ", ""],
    chunk_size=MAX_CHUNK_LENGTH,
    chunk_overlap=30,
)
md_splits = md_splitter.split_text(document)
text_splits = text_splitter.split_documents(md_splits)
len(text_splits)

43

In [28]:
text_splits[0]

Document(metadata={'H1': 'Definiowanie zadań modelowania', 'H2': 'Kluczowe elementy'}, page_content='# Definiowanie zadań modelowania  \n## Kluczowe elementy\n* Jakie jest zadanie biznesowe / domenowe\n* czy je dostatecznie dobrze rozumiemy\n* jakie są ograniczenia (czasowe, zasobów, formalne, ...)\n* Jakie są zadania modelowania')

In [29]:
text_splits[1]

Document(metadata={'H1': 'Definiowanie zadań modelowania', 'H2': 'Kluczowe elementy'}, page_content='* czy są odpowiednie dane do budowy modeli\n* czy rozumiemy proces, który je wygenerował\n* Jakie są kryteria sukcesu\n* na poziomie zadań modelowania (analityczne)\n* biznesowe')

In [31]:
split_lengths = [len(fragment.page_content) for fragment in text_splits]
print(max(split_lengths))
print(min(split_lengths))
print(sum(split_lengths) / len(split_lengths))

248
65
183.2093023255814
